In [1]:
import os
import geopandas as gpd

In [2]:
enriched_list = os.listdir(r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp")

In [3]:
point_layers = []
line_layers = []
poly_layers = []
for f in enriched_list:
    f_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\{}".format(f)
    gdb_layers = gpd.list_layers(f_path)
    for i in range(len(gdb_layers)):
        if 'point' in gdb_layers.loc[i, 'geometry_type'].lower():
            point_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
        elif 'line' in gdb_layers.loc[i, 'geometry_type'].lower():
            line_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
        else:
            poly_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
            
enriched_layers = {'point': point_layers, 
                  'line': line_layers,
                  'polygon': poly_layers}

In [4]:
enriched_layers['polygon'].append({'gdb_path': r"D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb", 'layer_name': 'Timber_Industry_Spatial_20241130'})

In [5]:
enriched_layers

{'point': [{'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\tmp\\CNRA_2020_2025.gdb',
   'layer_name': 'CNRA_enriched_20250421_point'},
  {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\tmp\\NFPORS_2020_2025.gdb',
   'layer_name': 'NFPORS_20250407_point'},
  {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\tmp\\PFIRS_2020_2025.gdb',
   'layer_name': 'PFIRS_20250402'},
  {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\tmp\\Timber_Nonspatial_2020_2025.gdb',
   'layer_name': 'Timber_Nonspatial_20250221'}],
 'line': [{'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\tmp\\CalTRANS_2021_2023.gdb',
   'layer_name': 'CalTRANS_enriched_20250108'},
  {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\tmp\\CNRA_2020_2025.gdb',
   'layer_name': 'CNRA_enriched_20250421_line'}],
 'polygon': [{'gdb_pa

In [6]:
import sys
sys.path.append('../')
from process.append import get_enriched_features

In [7]:
enriched_polygons, enriched_lines, enriched_points = get_enriched_features(enriched_layers)

2025-04-25 15:10:48,751 INFO  [process.append_polygon]  --------------------------------------------------------------------------------
2025-04-25 15:10:48,752 INFO  [process.append_polygon]  Concatenate all polygon records
2025-04-25 15:10:48,752 INFO  [process.append_polygon]  Load GeoDataFrame from the layer 'BLM_enriched_20250425' in 'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\BLM_2020_2025.gdb' 
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_re

In [8]:
california_boundary = gpd.read_file(r'D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb', 
                                    driver='OpenFileGDB', 
                                    layer='California')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [9]:
import dask_geopandas
# some data need to be converted to multi-type again
from utils.save_gdf_to_gdb import save_gdf_to_gdb

In [11]:
%%time

append_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb"

for df, lyr_name in zip([enriched_polygons,enriched_lines,enriched_points], ["appended_poly","appended_line","appended_point"]):
    # init dask gdf for multithread clipping
    ddf = dask_geopandas.from_geopandas(df, npartitions=16)
    # clip to california bounds
    append_clipped = ddf.sjoin(california_boundary, how='inner', predicate='intersects').compute()
    save_gdf_to_gdb(append_clipped, append_path, lyr_name)

2025-04-25 15:16:31,932 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-04-25 15:16:31,932 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-04-25 15:16:31,936 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field index_right of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Shape_Length' to 'Shape_Length_1'
  ogr_write(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Shape_Area' to 'Shape_Area_1'
  ogr_write(
2025-04-25 15:16:38,965 INFO  [pyogrio._io           ]  Created 62,088 records
2025-04-25 15:16:41,714 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_poly
2025-04-25 15:18:08,996 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-04-25 15:18:08,996 INFO  [utils.save_gdf_to_gdb ]        Chec

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field index_right of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(
2025-04-25 15:18:24,821 INFO  [pyogrio._io           ]  Created 103,837 records
2025-04-25 15:18:27,854 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_line
2025-04-25 15:18:29,282 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-04-25 15:18:29,283 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-04-25 15:18:29,357 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field index_right of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(
2025-04-25 15:18:30,092 INFO  [pyogrio._io           ]  Created 14,341 records
2025-04-25 15:18:30,204 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_point


CPU times: total: 15min 45s
Wall time: 2min 34s


In [12]:
enriched_polygons = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_poly')
enriched_points = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_point')
enriched_lines = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_line')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [17]:
f_path =r'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\temp_tables'

In [67]:
from datetime import datetime

In [62]:
def get_activity_report(enriched_points, enriched_lines, enriched_polygons):
    append_all = pd.concat([enriched_lines, enriched_points, enriched_polygons])
    append_all = append_all[(append_all['COUNTS_TO_MAS'] == 'YES')]
    
    append_all.geometry = gpd.points_from_xy(append_all['LONGITUDE'],append_all['LATITUDE'])
    
    
    append_all = append_all[["AGENCY",
        "ADMINISTERING_ORG",
        "PRIMARY_OWNERSHIP_GROUP",
        "COUNTY",
        "REGION",
        "ACTIVITY_DESCRIPTION",
        "ACTIVITY_CAT",
        "BROAD_VEGETATION_TYPE",
        "ACTIVITY_STATUS",
        "ACTIVITY_QUANTITY",
        "ACTIVITY_UOM",
        "ACTIVITY_END",
        "Year_txt",
        "geometry"
    ]]
    
    # check if geometry is_valid
    # personally this is redundent, unless lat, lon is not valid, but that would throw an error in previous part
    append_all = append_all[append_all.is_valid]
    
    
    def get_entity_type(agency):
        if agency in ['CALEPA', 'CALSTA', 'CNRA', 'PARKS', 'California State Parks']:
            return 'State'
        if agency in ['DOD', 'DOI', 'USDA', 'US Department of Energy', 'NPS']:
            return 'Federal'
        if agency in ['Industrial Timber', 'Timber Companies', 'TIMBER']:
            return 'Timber Companies'
        else:
            return None
        
        
    append_all['ENTITY_TYPE'] = append_all['AGENCY'].apply(get_entity_type)
    
    return append_all

In [63]:
activity_report_gdf = get_activity_report(enriched_points, enriched_lines, enriched_polygons)

In [69]:
save_gdf_to_gdb(activity_report_gdf, 
                r'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb', 
                f'activity_report{datetime.today().strftime('%Y%m%d')}')

2025-04-25 15:42:46,650 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-04-25 15:42:46,650 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-04-25 15:42:46,661 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb


2025-04-25 15:42:47,998 INFO  [pyogrio._io           ]  Created 121,153 records
2025-04-25 15:42:48,713 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb activity_report20250425


In [45]:
append_all = pd.concat([enriched_lines, enriched_points, enriched_polygons])
append_all = append_all[(append_all['COUNTS_TO_MAS'] == 'YES')]
for y in ['2021', '2022', '2023']:
    
    
    temp_report = append_all[append_all['Year_txt'] == y]
    temp_report.dropna(subset=['ACTIVITY_QUANTITY'])
    
    # table by admin org
    
    temp_report[['ADMINISTERING_ORG', 'ACTIVITY_QUANTITY']]\
        .groupby(['ADMINISTERING_ORG']).sum().to_csv(os.path.join(f_path, f'ADMINISTERING_ORG{y}.csv'))
    
    # table act cat
    temp_report[['ACTIVITY_CAT', 'ACTIVITY_QUANTITY']]\
        .groupby(['ACTIVITY_CAT']).sum().to_csv(os.path.join(f_path, f'ACTIVITY_CAT{y}.csv'))
    
    # PRIMARY_OWNERSHIP_GROUP
    temp_report[['PRIMARY_OWNERSHIP_GROUP', 'ACTIVITY_QUANTITY']]\
        .groupby(['PRIMARY_OWNERSHIP_GROUP']).sum().to_csv(os.path.join(f_path, f'PRIMARY_OWNERSHIP_GROUP{y}.csv'))
    
    # COUNTY
    temp_report[['COUNTY', 'ACTIVITY_QUANTITY']]\
        .groupby(['COUNTY']).sum().to_csv(os.path.join(f_path, f'COUNTY{y}.csv'))
    